<a href="https://colab.research.google.com/github/lejimmy/terra-python-sdk/blob/main/notebooks/Terra_SDK_Tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Luna to bLuna Arbitrager

In this notebook, we will create a simple Luna to bLuna arbitrager on the testnet.

Once you're comfortable with how the mechanics work, you can connect to the mainnet.

Remember to point your wallet and contract addresseses to the mainnet contracts.

## Install Python Terra SDK

We can install the Terra SDK for Python via the `pip` package manager.

In [1]:
# install terra sdk for python
!pip install terra-sdk

     |████████████████████████████████| 68 kB 2.9 MB/s 
     |████████████████████████████████| 1.3 MB 27.2 MB/s 
     |████████████████████████████████| 49 kB 4.1 MB/s 
     |████████████████████████████████| 61 kB 5.0 MB/s 
     |████████████████████████████████| 104 kB 39.2 MB/s 
     |████████████████████████████████| 142 kB 48.5 MB/s 
     |████████████████████████████████| 294 kB 45.7 MB/s 
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Next, we will import the modules needed for our project. 

In [2]:
# import the Terra Python SDK modules
from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.auth import StdFee
from terra_sdk.core.coins import Coins
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.wasm.msgs import dict_to_b64
from terra_sdk.core.wasm.msgs import MsgExecuteContract

## Connect to a Terra Node
In this section, we will connect to the `tequila-0004` test node and create a wallet.

In [3]:
# connect to test net node by specifying the url and chain-id
terra = LCDClient('https://tequila-lcd.terra.dev', 'tequila-0004')

In [4]:
# recover an existing wallet
mk = MnemonicKey(mnemonic='secret globe hint gather move evil monster column disorder burden couple odor salad holiday small float endorse alley escape veteran cruel cross web during')

In [5]:
# view wallet address
mk.acc_address

'terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx'

In [6]:
# create a wallet from the mnemonic key
wallet = terra.wallet(mk)

## Contract Addresses
The first contract address is for the Terraswap bLuna-Luna pair.

Since bLuna is not a native token, we will also need the contract address of the bLuna contract address when swapping back to Luna.

In [7]:
# terraswapblunaLunaPair address

# testnet 
bluna_luna_pair_address = 'terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v'

# mainnet 
# contract = 'terra1jxazgm67et0ce260kvrpfv50acuushpjsz2y0p'

In [8]:
# bluna contract addresses

# testnet
bluna_contract = 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x'

# mainnet
# bluna_contract = 'terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp'

In [ ]:
# swap amount in luna
swap_amount = 10

In [ ]:
# convert swap_amount to uluna
swap_amount = swap * 1000000

## Query bLuna-Luna Pair
Next, we'll query the contract to determine the current market price of the pair.

In [ ]:
# query bluna price
query_msg = {'simulation': {'offer_asset': {'amount': str(swap_amount), 'info': {'native_token': {'denom': 'uluna'}}}}}

In [ ]:
# query results
result = terra.wasm.contract_query(bluna_luna_pair_address, query_msg)
result

{'commission_amount': '33684',
 'return_amount': '11194488',
 'spread_amount': '4417'}

In [ ]:
# parse return amount
return_amount = int(result['return_amount'])
return_amount

11195463

In [ ]:
# belief price
belief_price = return_amount / swap_amount
belief_price

1.1195463

For a 10 Luna swap, we expect to receive ~1.12 bLuna for each Luna. In otherwords, bLuna is trading at a 12% discount.

## Execute Contract

Next, we will create a function to execute a message contract, sign the transaction, and broadcast the transaction.

In [ ]:
def execute_contract(sender, contract_addr, execute_msg, coins=None):
  execute = MsgExecuteContract(
      sender=sender.key.acc_address,
      contract=contract_addr,
      execute_msg=execute_msg,
      coins=coins
  )

  tx = sender.create_and_sign_tx(
      msgs=[execute], fee=StdFee(1000000, Coins(uluna=1000000))
  )

  result = terra.tx.broadcast(tx)
  return result

The following cells will send some amount of Luna from our wallet, to the Terraswap bLuna-Luna pair, sign the transaction, and broadcast the transaction.

In [ ]:
luna_to_bluna_msg = {
  "swap": {
    "offer_asset": {
      "info": {
        "native_token": {
          "denom": "uluna"
        }
      },
      "amount": str(swap_amount)
    },
    "to": mk.acc_address
  }
}

In [ ]:
# swap luna to bluna
result = execute_contract(wallet, bluna_luna_pair_address, luna_to_bluna_msg, Coins(uluna=swap_amount))
result

BlockTxBroadcastResult(height=5501367, txhash='3A148BA3EEA54622716CA93FF5C81CC868937DBACFF1094F141954F233AB84C2', raw_log='[{"msg_index":0,"log":"","events":[{"type":"execute_contract","attributes":[{"key":"sender","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"contract_address","value":"terra1ac24j6pdxh53czqyrkr6ygphdeftg7u3958tl2"},{"key":"sender","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"contract_address","value":"terra1ac24j6pdxh53czqyrkr6ygphdeftg7u3958tl2"},{"key":"sender","value":"terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v"},{"key":"contract_address","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"contract_address","value":"terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v"}]},{"type":"from_contract","attributes":[{"key":"contract_address","value":"terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v"},{"key":"action","value":"swap"},{"key":"offer_asset","value":"u

To reverse the swap, we will send some amount of bLuna to the Terraswap bLuna-Luna pair, and set a belief price for the swap.

In [ ]:
bluna_to_luna_msg = {
  "send": {
    "contract": "terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v",
    "amount": str(swap_amount),
    "msg": dict_to_b64({
      "swap": {
        "belief_price": str(belief_price),
        "max_spread": "0.1"
      }
    })
  }
}

In [ ]:
# swap bluna to luna
result_luna_to_bluna = execute_contract(wallet, bluna_contract, bluna_to_luna_msg)
result_luna_to_bluna

BlockTxBroadcastResult(height=5501369, txhash='D00F6A00BB5977DC0C2C6C0C5A12907C6008628DDC92F030806A649190AF36C5', raw_log='[{"msg_index":0,"log":"","events":[{"type":"execute_contract","attributes":[{"key":"sender","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"contract_address","value":"terra1ac24j6pdxh53czqyrkr6ygphdeftg7u3958tl2"},{"key":"sender","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"contract_address","value":"terra1ac24j6pdxh53czqyrkr6ygphdeftg7u3958tl2"},{"key":"sender","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"contract_address","value":"terra13e4jmcjnwrauvl2fnjdwex0exuzd8zrh5xk29v"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"contract_address","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"}]},{"type":"from_contract","attributes":[{"key":"contract_address","value":"terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x"},{"key":"action","value":"send"},{"key":"from","value":"terra1la

# Next Steps
Now that the contract integrations are working, the following next steps can extend the contracts:


*   Buy and sell signals
*   Submit bLuna to be slow burned
*   Deploy to an always on Replit server

